In [1]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2


In [7]:
frame_path = Path("frames")

# for frame in frame_path.iterdir():
#     img = cv2.imread(str(frame))
    
#     cv2.imshow("frame", img)
#     cv2.waitKey(0)

# cv2.destroyAllWindows()

In [8]:
def load_frames(frame_path ,_from = 0, _to = None):
    """
    Load frames from the frame_path directory
    _from: int, starting index
    _to: int, ending index
    
    return: list of frames
    
    if _to is None, load all frames from _from to the end
    """
    if not frame_path.exists():
        raise FileNotFoundError("Frame path does not exist")
    
    if not frame_path.is_dir():
        raise NotADirectoryError("Frame path is not a directory")
    
    if _from < 0:
        raise ValueError("Starting index must be greater than or equal to 0")
    
    if _to is not None and _to < _from:
        raise ValueError("Ending index must be greater than or equal to the starting index")
    
    frames = []
    for i, frame in enumerate(frame_path.iterdir()):
        if i < _from:
            continue
        if _to is not None and i > _to:
            break
        img = cv2.imread(str(frame))
        frames.append(img)
    return frames

In [10]:
frames = load_frames()

print(len(frames1), len(frames2))

5 6


In [11]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")  # Load the pre-trained model

100%|██████████| 21.5M/21.5M [00:10<00:00, 2.12MB/s]
